**Поиск экстремумов функции:**

In [1]:
from scipy.optimize import minimize
import numpy as np
e = 1e-10 # Очень близко к 0
fun = lambda x : (x[0]**2 + x[1]**2 ) 
cons = ({'type': 'eq', 'fun': lambda x: x[0]**4 + x[1]**4 - 1}, # x^2 + y^2 + z^21
        {'type': 'ineq', 'fun': lambda x: x[0] - e}, # x> = e эквивалентно x> 0
        {'type': 'ineq', 'fun': lambda x: x[1] - e},
       )
x0 = np.array((1.0, 1.0)) # Установить начальное значение
res = minimize(fun, x0, method='SLSQP', constraints=cons)
print('Максимальное значение:',res.fun)
print('Оптимальное решение:',res.x)
print('Успешно ли завершение итерации:', res.success)
print('Причина прекращения итерации:', res.message)

Максимальное значение: 1.414213562643511
Оптимальное решение: [0.84089642 0.84089642]
Успешно ли завершение итерации: True
Причина прекращения итерации: Optimization terminated successfully


$ \large \frac{1}{\sqrt[4]{2}}  = 0.84 $

In [2]:
!pip install ortools

In [3]:
import math
import time
import random
from copy import deepcopy
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [4]:
def create_data_model():
    data = {}
    data["distance_matrix"] = [[0, 4, 5, 7, 5],
                               [8, 0, 5, 6, 6],
                               [3, 5, 0, 9, 6],
                               [3, 5, 6, 0, 2],
                               [6, 2, 3, 8, 0]
    ]
    data["num_vehicles"] = 1
    data["depot"] = 0
    return data


def print_solution(manager, routing, solution):
    print(f"Objective: {solution.ObjectiveValue()} miles")
    index = routing.Start(0)
    plan_output = "Route for vehicle 0:\n"
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += f" {manager.IndexToNode(index)} ->"
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += f" {manager.IndexToNode(index)}\n"
    print(plan_output)
    plan_output += f"Route distance: {route_distance}miles\n"


def main():
    data = create_data_model()

    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )

    routing = pywrapcp.RoutingModel(manager)


    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )

    solution = routing.SolveWithParameters(search_parameters)

    if solution:
        print_solution(manager, routing, solution)


if __name__ == "__main__":
    main()

Objective: 18 miles
Route for vehicle 0:
 0 -> 1 -> 3 -> 4 -> 2 -> 0



In [5]:
def compute_way_cost(matrix, path):
    total_cost = 0
    for i in range(0, len(path)-1):
        first_city = path[i]
        second_city = path[i+1]
        if matrix[first_city][second_city] == '*':
            return False
        total_cost += matrix[first_city][second_city]
    first_city = path[-1]
    second_city = path[0]
    if matrix[first_city][second_city] == "*":
        return False
    total_cost += matrix[first_city][second_city]
    return total_cost


def get_new_way(path):
    new_way = deepcopy(path)
    first = random.randint(0, len(path)-1)
    second = random.randint(0, len(path)-1)
    while second == first:
        second = random.randint(0, len(path)-1)

    new_way[first], new_way[second] = new_way[second], new_way[first]
    return new_way

                            
def simulated_annealing(matrix, t_0=1000.0, t_min=0.005):
    length = len(matrix)
    template = list(range(0, length))

    global start_time
    start_time = time.time()
    global_min_cost = False
    
    while not global_min_cost:
        random.shuffle(template)
        global_min_cost = compute_way_cost(matrix, template)
        global_min_way = template
    t_k = t_0
    k = 1
    current_way = template
    current_way_cost = compute_way_cost(matrix, template)

    while t_k > t_min:
        t_k = t_0/(1+k)
        k += 1
                                              
        cost = False
        while not cost:
            new_way = get_new_way(current_way)
            cost = compute_way_cost(matrix, new_way)
        dcost = cost - current_way_cost
        if dcost <= 0:
            current_way_cost = cost
            current_way = new_way
        else:
            change_prob = math.exp(-dcost/t_k)
            random_point = random. random()
            if random_point > change_prob:
                continue
            else:
                current_way_cost = cost
                current_way = new_way

        if current_way_cost < global_min_cost and current_way_cost != False:
            global_min_cost = current_way_cost
            global_min_way = current_way
                                              
    return global_min_way, global_min_cost, k, matrix

In [6]:
matrix = [['*', 4, 5, 7, 5],
          [8, '*', 5, 6, 6],
          [3, 5, '*', 9, 6],
          [3, 5, 6, '*', 2],
          [6, 2, 3, 8, '*']
]

a, b, c, m = simulated_annealing(matrix, t_0 = 100, t_min = 0.005)
seconds = time.time() - start_time
a = '-'.join([str(i+1) for i in a])+f'-{str(a[0]+1)}'
print('Алгоритм иммитации отжига:')
print('Кол-во итераций', c)
print('Время выоплнения', seconds, 'секунд')
print('Маршрут обхода', a)
print('Стоимость', round(b,1))

Алгоритм иммитации отжига:
Кол-во итераций 20000
Время выоплнения 0.10748171806335449 секунд
Маршрут обхода 4-5-3-1-2-4
Стоимость 18


**При увеличении начальной температуры, в точности не проиграем, увеличится время выполнения.**

**При увеличении минимальнной температуры отжига, с большей вероятностью будем отлавливать неверный результат.**